In [1]:
%load_ext autoreload
%autoreload 2
import time
import tensorflow as tf
import p8_util

Using TensorFlow backend.


In [1]:
OUTPUT_DIR = './tmp/baseline'
datadir = './data'
is_tensorboard = True

In [2]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6006 &'
        .format(OUTPUT_DIR)
    )


    get_ipython().system_raw('./assets/ngrok http 6006 &')

In [4]:
# this will kill the processes for Tensorboard
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill
# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill    

In [5]:
import p8_util
filename_dataset=datadir+'/arr_keras_X_y_train_test.dump'
x_train, x_test, y_train, y_test, nClasses, tuple_dimension = p8_util.load_dataset(filename_dataset)

p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump
Dimensions= ((414, 224, 224, 3), (47, 224, 224, 3), (414,), (47,))
Number of classes= 3


In [6]:
import keras

LEARNING_RATE = 1.e-2
nn_type = 'CNN'
nb_class = nClasses
my_feature_columns, loss_reduction, tf_head = p8_util.get_tf_head("images",tuple_dimension, nClasses)
feature_columns = my_feature_columns

optimizer=tf.train.RMSPropOptimizer(learning_rate=LEARNING_RATE)
#optimizer = tf.train.AdagradOptimizer(learning_rate=LEARNING_RATE)
#optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, decay=1e-6, momentum=0.9, nesterov=True)
layer_size = 50
num_layers = 1
learn_mixture_weights = False
dropout = 0.5
seed=p8_util.RANDOM_SEED
is_cnn_batch_norm = True
cnn_layer_config={'feature_map_size':[128]}
initialiser_name = 'xavier'
oNNAdaNetBuilder = p8_util._NNAdaNetBuilder(nn_type, nb_class,feature_columns, optimizer, layer_size, num_layers,
               learn_mixture_weights, dropout, seed, is_cnn_batch_norm,cnn_layer_config, initialiser_name)


*** _NNAdaNetBuilder : Classes=3


In [7]:
#help(tf.train.get_global_step)

In [10]:
# Parameters that will be provided to model_fn
BATCH_SIZE = 40  #@param {type:"integer"}
MAX_STEPS = 200
params = {'learning_rate':LEARNING_RATE, 'net_builder':oNNAdaNetBuilder}
NN_TYPE = 'CNN'
my_config = p8_util.make_config(NN_TYPE,output_dir=OUTPUT_DIR, is_restored=False)  
NUM_EPOCHS = 50
classifier = tf.estimator.Estimator(model_fn=p8_util.my_model_fn, params=params, config=my_config)
start_time = time.time()
results, _ = tf.estimator.train_and_evaluate(
    classifier,
    train_spec=tf.estimator.TrainSpec(
        input_fn=p8_util.input_fn("train", x_train, y_train, NUM_EPOCHS\
                          , tuple_dimension=tuple_dimension\
                          , batch_size=BATCH_SIZE),
        max_steps=MAX_STEPS),
    
    eval_spec=tf.estimator.EvalSpec(
        input_fn=p8_util.input_fn("test", x_test, y_test, NUM_EPOCHS\
                           , tuple_dimension=tuple_dimension\
                           , batch_size=BATCH_SIZE),
        steps=None,
        throttle_secs=1))
end_time = time.time()

INFO:tensorflow:Using config: {'_model_dir': './tmp/baseline/CNN', '_tf_random_seed': 42, '_save_summary_steps': 5, '_save_checkpoints_steps': 5, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6d477e6048>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluat

INFO:tensorflow:Starting evaluation at 2019-04-17-12:12:07
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/baseline/CNN/model.ckpt-35
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

*** generator() : labels shape= (47,) / label values= 1
INFO:tensorflow:Finished evaluation at 2019-04-17-12:12:10
INFO:tensorflow:Saving dict for global step 35: eval_accuracy = 0.38297874, global_step = 35, loss = 221.13373
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 35: ./tmp/baseline/CNN/model.ckpt-35
INFO:tensorflow:Saving checkpoints for 40 into ./tmp/baseline/CNN/model.ckpt.
INFO:tensorflow:Calling model_fn.

*** _build_cnn_subnetwork() : width=224 / Heigh=224 / Channel=3
*** _build_cnn_subnetwork() : CNN layer size=50 / CNN layer= 1


*** INFO : accuracy= (<tf.Tensor 'accuracy/value:0' shape=() dtype=float32>, <tf.Tensor 'accuracy/update_op:0' shape=() dtype=float32>)
INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Starting evaluation at 2019-04-17-12:17:16
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/baseline/CNN/model.ckpt-75
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

*** generator() : labels shape= (47,) / label values= 1
INFO:tensorflow:Finished evaluation at 2019-04-17-12:17:19
INFO:tensorflow:Saving dict for global step 75: eval_accuracy = 0.25531915, global_step = 75, loss = 2097.4414
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 75: ./tmp/baseline/CNN/model.ckpt-75
INFO:tensorflow:Saving checkpoints for 80 into ./tmp/baseline/CNN/model.ckpt.
INFO:tensorflow:Calling model_fn.

*** _build_cnn_subnetwork() : width=224 / Heigh=224 / Channel=3
*** _build_cnn_subnetwork() : CNN layer size=50 / CNN layer= 1


*** INFO : accuracy= (<tf.Tensor 'accuracy/value:0' shape=() dtype=float32>, <tf.Tensor 'accuracy/update_op:0' shape=() dtype=float32>)
INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-17-12:22:27
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/baseline/CNN/model.ckpt-115
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

*** generator() : labels shape= (47,) / label values= 1
INFO:tensorflow:Finished evaluation at 2019-04-17-12:22:30
INFO:tensorflow:Saving dict for global step 115: eval_accuracy = 0.34042552, global_step = 115, loss = 47986.258
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 115: ./tmp/baseline/CNN/model.ckpt-115
INFO:tensorflow:Saving checkpoints for 120 into ./tmp/baseline/CNN/model.ckpt.
INFO:tensorflow:Calling model_fn.

*** _build_cnn_subnetwork() : width=224 / Heigh=224 / Channel=3
*** _build_cnn_subnetwork() : CNN layer size=50 / CNN layer= 1


*** INFO : accuracy= (<tf.Tensor 'accuracy/value:0' shape=() dtype=float32>, <tf.Tensor 'accuracy/update_op:0' shape=() dtype=floa

INFO:tensorflow:Starting evaluation at 2019-04-17-12:29:26
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/baseline/CNN/model.ckpt-155
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

*** generator() : labels shape= (47,) / label values= 1
INFO:tensorflow:Finished evaluation at 2019-04-17-12:29:31
INFO:tensorflow:Saving dict for global step 155: eval_accuracy = 0.34042552, global_step = 155, loss = 12043.8545
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 155: ./tmp/baseline/CNN/model.ckpt-155

*** generator() : labels shape= (414,) / label values= 0
INFO:tensorflow:Saving checkpoints for 160 into ./tmp/baseline/CNN/model.ckpt.
INFO:tensorflow:Calling model_fn.

*** _build_cnn_subnetwork() : width=224 / Heigh=224 / Channel=3
*** _build_cnn_subnetwork() : CNN layer size=50 / CNN layer= 1


*** INFO : accuracy= (<tf.Tensor 'accuracy/value:0' shape=() dtype=float32>, <tf.Tensor 'accuracy/update_op:0'

INFO:tensorflow:Starting evaluation at 2019-04-17-12:36:35
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/baseline/CNN/model.ckpt-195
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

*** generator() : labels shape= (47,) / label values= 1
INFO:tensorflow:Finished evaluation at 2019-04-17-12:36:38
INFO:tensorflow:Saving dict for global step 195: eval_accuracy = 0.34042552, global_step = 195, loss = 8733.091
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 195: ./tmp/baseline/CNN/model.ckpt-195

*** generator() : labels shape= (414,) / label values= 0
INFO:tensorflow:Saving checkpoints for 200 into ./tmp/baseline/CNN/model.ckpt.
INFO:tensorflow:Calling model_fn.

*** _build_cnn_subnetwork() : width=224 / Heigh=224 / Channel=3
*** _build_cnn_subnetwork() : CNN layer size=50 / CNN layer= 1


*** INFO : accuracy= (<tf.Tensor 'accuracy/value:0' shape=() dtype=float32>, <tf.Tensor 'accuracy/update_op:0' s

In [ ]:
print("\n------------------------------------------------")
print("Time (sec)", end_time-start_time)
for key in results.keys() :
    title =key.upper()
    print("{}: {}".format(title, results[key]))


In [ ]:
#help(tf.estimator.EstimatorSpec)

In [ ]:
OUTPUT_DIR